<a href="https://colab.research.google.com/github/Mohamed-Harish-M-123/Capstone-Project-IV/blob/main/Online_Retail_Customer_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Project Name: Online Retail Customer Segmentation**
####**Project Type:** Unsupervised ML
####**Contribution:** Individual
####**Name:** MOHAMED HARISH

###**GitHub link:** https://github.com/Mohamed-Harish-M-123/Capstone-Project-IV

##**Business context:**
####In this project, our task is to identify major customer segments on a transactional data set which contains all the transactions occuring between 01/12/2010 and 09/12/2011 for a UK based and registered non-store online retail. The company mainly sells unique all-occasion gifts. Many customer of the company are wholesalers.

##**Problem statement:**
####The problem at hand is to develop an effective online retail customer segmentation model using unsupervised machine learning techniques. The goal is to identify distinct groups of customers based on their purchasing behaviors, preferences, and demographic information, in order to enable personalized marketing strategies and enhance the overall customer experience.

###**Objective:**
####The objective of this project is to develop a customer segmentation model that can effectively group customers based on their purchasing behavior and demographic attributes, enabling the online retailer to personalize marketing campaigns, optimize product recommendations, and improve customer satisfaction. By understanding the distinct needs and preferences of different customer segments, the retailer can tailor their strategies to enhance customer engagement, increase sales, and drive long-term customer loyalty.

###**Importing libraries**

In [1]:
# Import required libraries for this project

from google.colab import drive

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
